In [ ]:
import ROOT

# Introduction to RDataFrame


## ROOT RDataFrame

[RDataFrame documentation](https://root.cern/doc/master/classROOT_1_1RDataFrame.html)

- RDF is ROOT's high-level analysis interface. 

- Users define their analysis as a sequence of operations to be performed on the data-frame object; 

    - the framework takes care of the management of the loop over entries as well as low-level details such as I/O and parallelisation.

- RDataFrame provides methods to perform most common operations required by ROOT analyses: 

    - at the same time, users can just as easily specify custom code that will be executed in the event loop.
<img src="../images/rdf_1.png" style="width: 75%;">

## Chapter 1: Basics of HEP data analysis with RDataFrame
RDataFrame allows reading and writing trees, aiming at making HEP analysis easy to write and fast to perform.

In [ ]:
treename = "dataset"
filename = "../data/example_file.root"
df = ROOT.RDataFrame(treename, filename)

print(f"Columns in the dataset: {df.GetColumnNames()}")

Now we can `Define` new quantities, `Filter` rows based on custom expressions and retrieve some data aggregations such as a `Count` and a `Mean`:

In [ ]:
def1 = df.Define("c", "a+b")

fil1 = def1.Filter("c < 0.5")

count = fil1.Count()
mean = fil1.Mean("c")
display = fil1.Display(["a","b","c"])

print(f"Number of rows after filter: {count.GetValue()}")
print(f"Mean of column c after filter: {mean.GetValue()}")
print("Dataset contents:")
display.Print()

## Histograms with RDataFrame
RDataFrame helps you streamline the creation and filling of histogram objects from your data. 

For example:

In [ ]:
%jsroot on
c = ROOT.TCanvas()
h = df.Histo1D("vec1")
h.Draw()
c.Draw()

- `Histo1D` will create a one-dimensional histogram holding `double` values. 

- `Histo{2,3}D` do the same in higher dimensions. 

- These operations also accept a tuple with the same arguments that would be passed to the equivalent histogram object constructors. 

- For example:

In [ ]:
histo_name = "histo_name"
histo_title = "histo_title"
nbinsx = 100
xlow = -10
xup = 10

# The traditional TH1D constructor
# ROOT.TH1D(histo_name, histo_title, nbinsx, xlow, xup)

# With RDataFrame
c = ROOT.TCanvas()
h = df.Histo1D((histo_name, histo_title, nbinsx, xlow, xup), "vec1")
h.Draw()
c.Draw()

## Think about data-flow
RDataFrame is built with a modular and flexible workflow in mind, summarised as follows:

* build a data-frame object by specifying your data-set
* apply a series of transformations to your data
  * filter (e.g. apply some cuts) or
  * define a new column (e.g. the result of an expensive computation on columns)
* apply actions to the transformed data to produce results (e.g. fill a histogram)

### Important Note!
Make sure to **book all transformations and actions before** you access the contents of any of the results: this lets RDataFrame accumulate work and then produce all results at the same time, upon first access to any of them.

In [ ]:
df_wrong = ROOT.RDataFrame(treename, filename)

h_a = df_wrong.Histo1D("a")
h_a_val = h_a.GetValue()

h_b = df_wrong.Histo1D("b")
h_b_val = h_b.GetValue()

h_vec1 = df_wrong.Histo1D("vec1")
h_vec1_val = h_vec1.GetValue()

print(f"The dataset was processed {df_wrong.GetNRuns()} times.")


In [ ]:
df_good = ROOT.RDataFrame(treename, filename)

h_a = df_good.Histo1D("a")
h_b = df_good.Histo1D("b")
h_vec1 = df_good.Histo1D("vec1")

h_a_val = h_a.GetValue()
h_b_val = h_b.GetValue()
h_vec1_val = h_vec1.GetValue()

print(f"The dataset was processed {df_good.GetNRuns()} time.")

## Operation categories in RDataFrame
There are 3 main types of operations you can perform on RDataFrames:

**Transformations**: manipulate the dataset, return a modified RDataFrame for further processing.

| Transformation    | Description                                                |
|-------------------|------------------------------------------------------------|
| Alias()           | Introduce an alias for a particular column name.           |
| Define()          | Creates  a new column in the dataset.                      |
| Filter()          | Filter rows based on user-defined conditions.              |

**Actions**: aggregate (parts of) the dataset into a result.

| Action                        | Description                                                                          |
|------------------------------------|--------------------------------------------------------------------------------------|
| Count()                            | Return the number of events processed.                                               |
| Display()                          | Provides a printable object representing the dataset contents.                       |
| Graph()                            | Fills a TGraph  with the two columns provided.                                       |
| Histo1D(), Histo2D(), Histo3D()    | Fill a one-, two-, three-dimensional histogram with the processed column values.     |
| Max(), Min()                       | Return the maximum(minimum) of processed column values.                              |
| Snapshot()        | Writes processed data-set to a new TTree.              |
| ...                                | ...  

**Queries**: these methods  query information about your dataset and the RDataFrame status.

| Operation           | Description                                                                              |
|---------------------|------------------------------------------------------------------------------------------|
| GetColumnNames()    | Get the names of all the available columns of the dataset.                               |
| GetColumnType()     | Return the type of a given column as a string.                                           |
| SaveGraph()         | Export the computation graph of an RDataFrame in graphviz format for easy inspection.     |
| ...                 | ...                                                                                      |

For the full list, visit [RDataFrame documentation page](https://root.cern/doc/master/classROOT_1_1RDataFrame.html#cheatsheet).

# Chapter 2: Collections

## Working with `numpy` arrays
- RDataFrame offers interoperability with `numpy` arrays. 

- It can be created from a dictionary of such arrays and it can also export its contents to the same format. 

- All operations are available also when using the `numpy`-based dataset.

- **Note:** this support is limited to one-dimensional numpy arrays, which are directly mapped to columns in the RDataFrame.

In [ ]:
import numpy

np_dict = {colname: numpy.random.rand(100) for colname in ["a","b","c"]}

df = ROOT.RDF.FromNumpy(np_dict)

print(f"Columns in the RDataFrame: {df.GetColumnNames()}")

In [ ]:
co = df.Count()
m_a = df.Mean("a")

fil1 = df.Filter("c < 0.7")
def1 = fil1.Define("d", "a+b+c")
h = def1.Histo1D("d")

c = ROOT.TCanvas()
h.Draw()

print(f"Number of rows in the dataset: {co.GetValue()}")
print(f"Average value of column a: {m_a.GetValue()}")
c.Draw()

## Collections and object selections

- RDataFrame reads collections as the special type [ROOT::RVec](https://root.cern/doc/master/classROOT_1_1VecOps_1_1RVec.html) - e.g. a branch containing an array of floating point numbers can be read as a `ROOT::RVec<float>`.

- C-style arrays (with variable or static size), `std::vectors` and many other collection types can be read this way. 

- When reading ROOT data, column values of type `ROOT::RVec<T>` perform no copy of the underlying array.

- `RVec` is a container similar to `std::vector` (and can be used just like a `std::vector`) but it also offers a rich interface to operate on the array elements in a vectorised fashion, similarly to Python's NumPy arrays.

In [ ]:
treename = "myDataset"
filename = "../data/collections_dataset.root"
df = ROOT.RDataFrame(treename, filename)

print(f"Columns in the dataset: {df.GetColumnNames()}")

To quickly inspect the data we can export it as a dictionary of `numpy` arrays thanks to the `AsNumpy` RDataFrame method. 

Note that for each row, `E` is an array of values:

In [ ]:
npy_dict = df.AsNumpy(["E"])

for row, vec in enumerate(npy_dict["E"]):
    print(f"\nRow {row} contains:\n{vec}")

### Define a new column with operations on RVecs

In [ ]:
df1 = df.Define("good_pt", "sqrt(px*px + py*py)[E>100]")

`sqrt(px*px + py*py)[E>100]`:
- `px`, `py` and `E` are the columns, the elements of those columns are `RVec`s

- Operations on `RVec`s, such as sum, product, sqrt, preserve the dimensionality of the array

- `[E>100]` selects the elements of the array that satisfy the condition

- `E > 100`: boolean expressions on `RVec`s such as `E > 100` return a mask, that is an array with information which values pass the selection (e.g. `[0, 1, 0, 0]` if only the second element satisfies the condition)

### Now we can plot the newly defined column values in a histogram

In [ ]:
c = ROOT.TCanvas()
h = df1.Histo1D(("pt", "pt", 16, 0, 4), "good_pt")
h.Draw()
c.Draw()

# Chapter 3: What about the results? 


## Snapshot - save datasets to a ROOT file after processing
With RDataFrame, you can read your dataset, add new columns with processed values and finally use `Snapshot` to save the resulting data to a ROOT file in TTree format.

In [ ]:
df = ROOT.RDataFrame("dataset","../data/example_file.root")
df1 = df.Define("c","a+b")

out_treename = "outtree"
out_filename = "outtree.root"
out_columns = ["a","b","c"]
snapdf = df1.Snapshot(out_treename, out_filename, out_columns)

We can now check that the dataset was correctly stored in a file:

In [ ]:
%%bash
rootls -lt outtree.root

Result of a Snapshot is still an RDataFrame that can be further used:

In [ ]:
snapdf.Display().Print()

## Cutflow reports
Filters applied to the dataset can be given a name. The `Report` method will gather information about filter efficiency and show the data flow between subsequent cuts on the original dataset.


In [ ]:
df = ROOT.RDataFrame("sig_tree", "https://root.cern/files/Higgs_data.root")

filter1 = df.Filter("lepton_eta > 0", "Lepton eta cut")
filter2 = filter1.Filter("lepton_phi < 1", "Lepton phi cut")

rep = df.Report()
rep.Print()

# Chapter 4: More advance features - what else would we need in a real analysis?

## RDatsetSpec and FromSpec

What if you have many different samples - multiple tree or rntuple names and many different file names with some metadata information as well? 
- We have the RDatasetSpec class for that!
- And especially, get familiar with the `FromSpec` method thanks to which you can create a single dataframe consisting of many different samples which are added via a JSON file.

Example:
```json
{
   "samples": {
      "sampleA": {
         "trees": ["tree1", "tree2"],
         "files": ["file1.root", "file2.root"],
         "metadata": {"lumi": 1.0, }
      },
      "sampleB": {
         "trees": ["tree3", "tree4"],
         "files": ["file3.root", "file4.root"],
         "metadata": {"lumi": 0.5, }
      },
      ...
    },
}
```

In [ ]:
df = ROOT.RDF.Experimental.FromSpec("../data/dataset_spec.json")

And now simply continue your analysis as per usual... 

But what if some operations are valid only for some of the samples? 
- Solution: use `DefinePerSample` method - the samples can be recognized thanks to one or more metadata instances and then we can make use of `RSampleInfo` class. 

In [ ]:
df = df.DefinePerSample("xsecs", 'rdfsampleinfo_.GetD("xsecs")') 
# This is a teaser - how this can be used further is shown in the last chapter of this notebook!

## Including systematic variations in your analysis

At some point in the development of a HEP data analysis workflow, the treatment of systematic variations in the observed quantities will become necessary. From the standpoint of a HEP physicist, the study of systematic variations involves many different, often conceptually complex cases. From the standpoint of the pure numerical computation, however, what typically happens is that the application must produce multiple results instead of a single one, each computed in a "universe" in which certain inputs take modified values.

In the next few cells, we explore the RDataFrame API devoted to helping the user include systematic variations in their analysis code.

In [ ]:
treename = "myDataset"
filename = "../data/collections_dataset.root"
df = ROOT.RDataFrame(treename, filename)

### Registering variations for one observable

As a basic example, let's see how to include two variations for the column `px`, including giving them special labels. Note that in the call to `Vary` we can use values from columns available in the dataset, including the column we are currently registering variations for. In this example, we are booking the nominal histogram via the usual `Histo1D` method.

Note the following in the example:

* Custom names for variations can be passed in a list via the `variationTags` parameter.
* The name of the input column is used as the default name for the variation, unless the `variationName` parameter is used as in this example.
* The full variation name will be composed of the varied column name and the variation tags (e.g. "mypt:down", "mypt:up" in this example).
* The histogram is filled with values of the `good_pt` column, defined after the `Vary` call. The presence of systematic variations for certain columns is automatically propagated through filters, defines and actions, and RDataFrame will take these dependencies into account when producing varied results.

In [ ]:
df = df.Define("pt", "sqrt(px*px + py*py)")

nominal_histo = (
    df.Vary(
        colName="pt",
        expression="ROOT::RVec<ROOT::RVecD>{pt*0.95, pt*1.05}",
        variationTags=["down", "up"],
        variationName="mypt")
      .Define("good_pt", "pt[E>100]")
      .Histo1D("good_pt")
)

In order to retrieve also all the varied histograms, we pass the pointer to the action just booked to the `VariationsFor` function, as shown below. This will return a dictionary containing the nominal histogram as well as all the varied histograms.

In [ ]:
all_histos = ROOT.RDF.Experimental.VariationsFor(nominal_histo)

c = ROOT.TCanvas()

all_histos["nominal"].SetLineColor(ROOT.kBlue)
all_histos["nominal"].Draw()

all_histos["mypt:down"].SetLineColor(ROOT.kRed)
all_histos["mypt:down"].Draw("SAME")

all_histos["mypt:up"].SetLineColor(ROOT.kGreen)
all_histos["mypt:up"].Draw("SAME")

c.Draw()

### Extra - registering variations for multiple columns simultaneously

The `Vary` function also allows to vary multiple columns simultaneously (in "lockstep"). The expression in this case must return an RVec of RVecs, one per column: each inner vector contains the varied values for one column, and the inner vectors follow the same ordering as the column names passed as first argument. Besides the variation tags, in this case we also have to explicitly pass a variation name as there is no one column name that can be used as default.

In [ ]:
df = ROOT.RDataFrame(treename, filename)

nominal_histo_lockstep = (
    df.Vary(
        colNames=["px", "py"],
        expression="ROOT::RVec<ROOT::RVec<ROOT::RVecD>>{{px*0.95, px*1.05}, {py*0.95, py*1.05}}",
        variationTags=["down", "up"],
        variationName="pxAndpy")
      .Define("pt_lockstep", "sqrt(px*px + py*py)[E>100]")
      .Histo1D("pt_lockstep")
)

In [ ]:
all_histos = ROOT.RDF.Experimental.VariationsFor(nominal_histo_lockstep)

c = ROOT.TCanvas()

all_histos["nominal"].SetLineColor(ROOT.kBlue)
all_histos["nominal"].Draw()

all_histos["pxAndpy:down"].SetLineColor(ROOT.kRed)
all_histos["pxAndpy:down"].Draw("SAME")

all_histos["pxAndpy:up"].SetLineColor(ROOT.kGreen)
all_histos["pxAndpy:up"].Draw("SAME")

c.Draw()

## Using C++ functions in Python
- We still want to perform complex operations in Python but plain Python code is prone to be slow and not thread-safe. 

- Instead, you can inject C++ functions that will do the work in your event loop during runtime. 

- This mechanism uses the C++ interpreter `cling` shipped with ROOT, making this possible in a single line of code. 

- Let's start by defining a function that will allow us to change the type of a the RDataFrame dataset entry numbers (stored in the special column "rdfentry") from `unsigned long long` to `float`.

In [ ]:
%%cpp

float asfloat(unsigned long long entrynumber){
    return entrynumber;
}

Then let's define another function that takes a `float` values and computes its square.

In [ ]:
%%cpp

float square(float val){
    return val * val;
}

And now let's use these functions with RDataFrame! 

We start by creating an empty RDataFrame with 100 consecutive entries and defining new columns on it:

In [ ]:
# Create a new RDataFrame from scratch with 100 consecutive entries
df = ROOT.RDataFrame(100)

# Create a new column using the previously declared C++ functions
df1 = df.Define("a", "asfloat(rdfentry_)")
df2 = df1.Define("b", "square(a)")

We can now plot the values of the columns in a graph:

In [ ]:
# Show the two columns created in a graph
c = ROOT.TCanvas()
graph = df2.Graph("a","b")
graph.SetMarkerStyle(20)
graph.SetMarkerSize(0.5)
graph.SetMarkerColor(ROOT.kBlue)
graph.SetTitle("My graph")
graph.Draw("AP")
c.Draw()

# Chapter 5: Scaling up your analysis - multi-threading and distributed RDataFrame

## Using all cores of your machine with multi-threaded RDataFrame
- RDataFrame can transparently perform multi-threaded event loops to speed up the execution of its actions. 

- Users have to call `ROOT::EnableImplicitMT()` before constructing the RDataFrame object to indicate that it should take advantage of a pool of worker threads. 

- Each worker thread processes a distinct subset of entries, and their partial results are merged before returning the final values to the user.

- RDataFrame operations such as Histo1D or Snapshot are guaranteed to work correctly in multi-thread event loops. 

- User-defined expressions, such as strings or lambdas passed to `Filter`, `Define`, `Foreach`, `Reduce` or `Aggregate` will have to be thread-safe, i.e. it should be possible to call them concurrently from different threads.

In [ ]:
%%time
# Activate multithreading capabilities
# By default takes all available cores on the machine
ROOT.EnableImplicitMT()

# treename = "Events"
# filename = "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root"
# df = ROOT.RDataFrame(treename, filename)

# df.Sum("nMuon").GetValue()

# Disable implicit multithreading when done
ROOT.DisableImplicitMT()

## Multiple concurrent RDataFrame runs
If your analysis needs multiple RDataFrames to run (for example multiple dataset samples, data vs simulation etc.), make use of `ROOT.RDF.RunGraphs` 

In [ ]:
ROOT.EnableImplicitMT()
treename1 = "myDataset"
filename1 = "../data/collections_dataset.root"
treename2 = "dataset"
filename2 = "../data/example_file.root"

df1 = ROOT.RDataFrame(treename1, filename1)
df2 = ROOT.RDataFrame(treename2, filename2)
h1 = df1.Histo1D("px")
h2 = df2.Histo1D("a")


ROOT.RDF.RunGraphs((h1, h2))
ROOT.DisableImplicitMT()

In [ ]:
c = ROOT.TCanvas()
h1.Draw()
c.Draw()

In [ ]:
c = ROOT.TCanvas()
h2.Draw()
c.Draw()

## Distributed RDataFrame

An `RDataFrame` analysis written in Python can be executed both *locally* - possibly in parallel on the cores of the machine - and *distributedly* by offloading computations to external resources, which include:

- [Spark](https://spark.apache.org/) and 
- [Dask](https://dask.org/) clusters. 

- This feature is enabled by the architecture depicted below.

- It shows that RDataFrame computation graphs can be mapped to different kinds of resources via backends.

- In this notebook we will exercise the Dask backend, which divides an `RDataFrame` input dataset in logical ranges and submits computations for each of those ranges to Dask resources.

<img src="../images/DistRDF_architecture.png" alt="Distributed RDataFrame">

### Create a Dask client

- In order to work with a Dask cluster we need a `Client` object.
- It represents the connection to that cluster and allows to configure execution-related parameters (e.g. number of cores, memory). 
- The client object is just the intermediary between our client session and the cluster resources, 
- Dask supports many different resource managers.
- We will follow the [Dask documentation](https://distributed.dask.org/en/stable/client.html) regarding the creation of a `Client`.

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=2, threads_per_worker=1, processes=True, memory_limit="2GiB")
client = Client(cluster)

### Create a ROOT dataframe

We now create an RDataFrame based on the same dataset seen in the exercise [rdataframe-dimuon](exercises/rdataframe-dimuon.ipynb).

A Dask `RDataFrame` receives two extra parameters: 
- the executor, which in this case will be the dask client object
- the number of partitions to apply to the dataset (npartitions)

Besides this detail, a Dask `RDataFrame` is not different from a local `RDataFrame`: the analysis presented in this notebook would not change if we wanted to execute it locally.

In [ ]:
# Use a Dask RDataFrame
df = ROOT.RDataFrame("h42",
                "https://root.cern/files/h1big.root",
                executor=client,
                npartitions=4)

### Run your analysis unchanged

- From now on, the rest of your application can be written **exactly** as we have seen with local RDataFrame. 

- The goal of the distributed RDataFrame module is to support all the traditional RDataFrame operations (those that make sense in a distributed context at least). 

- The list of operations that are currently available in distributed mode and can be found in the corresponding [section of the documentation](https://root.cern/doc/master/classROOT_1_1RDataFrame.html#rdf_distrdf)

# Extra Chapter: The Higgs to four lepton analysis from the ATLAS Open Data release of 2020, with RDataFrame

We have briefly discussed most of what is needed to follow this analysis - try it yourself and discover the Higgs boson!



This exercise is the Higgs to four lepton analysis from the ATLAS Open Data release in 2020 (http://opendata.atlas.cern/release/2020/documentation/). The data was taken with the ATLAS detector during 2016 at a center-of-mass energy of 13 TeV. The decay of the Standard Model Higgs boson to two Z bosons and subsequently to four leptons is called the "golden channel". The selection leads to a narrow invariant mass peak on top a relatively smooth and small background, revealing the Higgs at 125 GeV. Systematic errors for the MC scale factors are computed and the Vary function of RDataFrame is used for plotting. The analysis is translated to an RDataFrame workflow processing about 300 MB of simulated events and data.


In [ ]:
import os

## Specifying the input dataset

In this exercise, we use a JSON config file to define the dataset specification. It contains the names of the files, together with metadata associated with different samples. The metadata contained in the JSON file is accessible within a [`DefinePerSample`](https://root.cern/doc/v628/classROOT_1_1RDF_1_1RInterface.html#a29d77593e95c0f84e359a802e6836a0e) call, through the [`RSampleInfo`](https://root.cern/doc/master/classROOT_1_1RDF_1_1RSampleInfo.html) class.

In [ ]:
# The analysis needs custom code written in a separate header, so we include it here
ROOT.gInterpreter.Declare('#include "utils.h"')

dataset_spec = "../data/dataset_spec.json"
df = ROOT.RDF.Experimental.FromSpec(dataset_spec)  # Creates a single dataframe for all the samples

df = df.DefinePerSample("xsecs", 'rdfsampleinfo_.GetD("xsecs")')
df = df.DefinePerSample("lumi", 'rdfsampleinfo_.GetD("lumi")')
df = df.DefinePerSample("sumws", 'rdfsampleinfo_.GetD("sumws")')
df = df.DefinePerSample("sample_category", 'rdfsampleinfo_.GetS("sample_category")')
df = df.DefinePerSample("scale", "scale(rdfslot_, rdfsampleinfo_)")

## Perform selections

We need to select events with exactly four good leptons conserving charge and lepton numbers. Note that all collections read by RDataFrame are implicitly handled as the `ROOT::RVec` type.

In [ ]:
# Select electron or muon trigger
df = df.Filter("trigE || trigM")

# good_lep is the mask for the good leptons.
# The lepton types are PDG numbers and set to 11 or 13 for an electron or muon
# irrespective of the charge.

df = (
    df.Define(
        "good_lep",
        "abs(lep_eta) < 2.5 && lep_pt > 5000 && lep_ptcone30 / lep_pt < 0.3 && lep_etcone20 / lep_pt < 0.3",
    )
    .Filter("Sum(good_lep) == 4")
    .Filter("Sum(lep_charge[good_lep]) == 0")
    .Define("goodlep_sumtypes", "Sum(lep_type[good_lep])")
    .Filter("goodlep_sumtypes == 44 || goodlep_sumtypes == 52 || goodlep_sumtypes == 48")
)

# Apply additional cuts depending on lepton flavour
df = df.Filter(
    "GoodElectronsAndMuons(lep_type[good_lep], lep_pt[good_lep], lep_eta[good_lep], lep_phi[good_lep], lep_E[good_lep], lep_trackd0pvunbiased[good_lep], lep_tracksigd0pvunbiased[good_lep], lep_z0[good_lep])"
)

# Create new columns with the kinematics of good leptons
df = (
    df.Define("goodlep_pt", "lep_pt[good_lep]")
    .Define("goodlep_eta", "lep_eta[good_lep]")
    .Define("goodlep_phi", "lep_phi[good_lep]")
    .Define("goodlep_E", "lep_E[good_lep]")
    .Define("goodlep_type", "lep_type[good_lep]")
)

# Select leptons with high transverse momentum
df = df.Filter("goodlep_pt[0] > 25000 && goodlep_pt[1] > 15000 && goodlep_pt[2] > 10000")

## Definition of the relevant observables

In [ ]:
# Reweighting of the samples is different for "data" and "MC".
# Use DefinePerSample to define which samples are MC and hence need reweighting
df = df.DefinePerSample("isMC", 'rdfsampleinfo_.Contains("mc")')
df = df.Define(
    "weight",
    "double x; return isMC ? weights(scaleFactor_ELE, scaleFactor_MUON, scaleFactor_LepTRIGGER, scaleFactor_PILEUP, scale, mcWeight, xsecs, sumws, lumi) :  1.;",
)

df = df.Define("m4l", "ComputeInvariantMass(goodlep_pt, goodlep_eta, goodlep_phi, goodlep_E)")

# Book histograms for the four different samples: data, higgs, zz and other (this is specific to this particular analysis)
histos = []
for sample_category in ["data", "higgs", "zz", "other"]:
    histos.append(
        df.Filter(f'sample_category == "{sample_category}"').Histo1D(
            ROOT.RDF.TH1DModel(f"{sample_category}", "m4l", 24, 80, 170),
            "m4l",
            "weight",
        )
    )

## Evaluate systematic uncertainties

The systematic uncertainty in this analysis is the MC scale factor uncertainty that depends on lepton kinematics such as pT or pseudorapidity. Muons uncertainties are negligible, as stated in https://atlas.web.cern.ch/Atlas/GROUPS/PHYSICS/PAPERS/MUON-2018-03/. Electrons uncertainties are evaluated based on the plots available in https://doi.org/10.48550/arXiv.1908.00005. The uncertainties are linearly interpolated, using the `TGraph::Eval()` method, to cover a range of pT values covered by the analysis.


In [ ]:
# Create one helper object to run the interpolation for the systematic uncertainty evaluation in the event loop
ROOT.gInterpreter.ProcessLine("VaryHelper variationsFactory;")

In [ ]:
# Use the Vary method to add the systematic variations to the total MC scale factor ("weight") of the analysis.
df_variations_mc = (
    df.Filter("isMC == true")
    .Vary("weight", "variationsFactory(weight, goodlep_pt, goodlep_type)", ["up", "down"])
    .Histo1D(ROOT.RDF.TH1DModel("Invariant Mass", "m4l", 24, 80, 170), "m4l", "weight")
)
histos_mc = ROOT.RDF.Experimental.VariationsFor(df_variations_mc)

## Retrieving histograms and plotting

We reached the end of the analysis part. We now evaluate the total MC uncertainty based on the variations. No computation graph was triggered yet, we trigger the computation graph for all histograms at once now, by calling 'histos_mc["nominal"].GetXaxis()'. Note, in this case the uncertainties are symmetric.

In [ ]:
for i in range(0, histos_mc["nominal"].GetXaxis().GetNbins()):
    (
        histos_mc["nominal"].SetBinError(
            i, (histos_mc["weight:up"].GetBinContent(i) - histos_mc["nominal"].GetBinContent(i))
        )
    )

Make the plot of the data, individual MC contributions and the total MC scale factor systematic variations.

In [ ]:
ROOT.gROOT.SetStyle("ATLAS")

# Create canvas with pad
c4 = ROOT.TCanvas("c4", "", 620, 620)
pad = ROOT.TPad("upper_pad", "", 0, 0, 1, 1)
pad.SetTickx(False)
pad.SetTicky(False)
pad.Draw()
pad.cd()

# Draw stack with MC contributions
stack = ROOT.THStack()

# Retrieve values of the data and MC histograms in order to plot them.
# Draw cloned histograms to preserve graphics when original objects goes out of scope
# Note: GetValue() action operation is performed after all lazy actions of the RDF were defined first.
h_data = histos[0].GetValue().Clone()
h_higgs = histos[1].GetValue().Clone()
h_zz = histos[2].GetValue().Clone()
h_other = histos[3].GetValue().Clone()

for h, color in zip([h_other, h_zz, h_higgs], [ROOT.kViolet-9, ROOT.kAzure-9, ROOT.kRed+2]):
    h.SetLineWidth(1)
    h.SetLineColor(1)
    h.SetFillColor(color)
    stack.Add(h)

stack.Draw("HIST")
stack.GetXaxis().SetLabelSize(0.04)
stack.GetXaxis().SetTitleSize(0.045)
stack.GetXaxis().SetTitleOffset(1.3)
stack.GetXaxis().SetTitle("m_{4l}^{H#rightarrow ZZ} [GeV]")
stack.GetYaxis().SetLabelSize(0.04)
stack.GetYaxis().SetTitleSize(0.045)
stack.GetYaxis().SetTitle("Events")
stack.SetMaximum(35)
stack.GetYaxis().ChangeLabel(1, -1, 0)

# Draw MC scale factor and variations
histos_mc["nominal"].SetFillColor(ROOT.kBlack)
histos_mc["nominal"].SetFillStyle(3254)
h_nominal = histos_mc["nominal"].DrawClone("E2 same")
histos_mc["weight:up"].SetLineColor(ROOT.kGreen+2)
h_weight_up = histos_mc["weight:up"].DrawClone("HIST SAME")
histos_mc["weight:down"].SetLineColor(ROOT.kBlue+2)
h_weight_down = histos_mc["weight:down"].DrawClone("HIST SAME")

# Draw data histogram
h_data.SetMarkerStyle(20)
h_data.SetMarkerSize(1.2)
h_data.SetLineWidth(2)
h_data.SetLineColor(ROOT.kBlack)
h_data.Draw("E SAME")  # Draw raw data with errorbars

# Add legend
legend = ROOT.TLegend(0.57, 0.65, 0.94, 0.94)
legend.SetTextFont(42)
legend.SetFillStyle(0)
legend.SetBorderSize(0)
legend.SetTextSize(0.02)
legend.SetTextAlign(32)
legend.AddEntry(h_data, "Data", "lep")
legend.AddEntry(h_higgs, "Higgs MC", "f")
legend.AddEntry(h_zz, "ZZ MC", "f")
legend.AddEntry(h_other, "Other MC", "f")
legend.AddEntry(h_weight_down, "Total MC Variations Down", "l")
legend.AddEntry(h_weight_up, "Total MC Variations Up", "l")
legend.AddEntry(h_nominal, "Total MC Uncertainty", "f")
legend.Draw()

text = ROOT.TLatex()
text.SetTextFont(72)
text.SetTextSize(0.04)
text.DrawLatexNDC(0.19, 0.85, "ATLAS")
text.SetTextFont(42)
text.DrawLatexNDC(0.19 + 0.15, 0.85, "Open Data")
text.SetTextSize(0.035)
text.DrawLatexNDC(0.21, 0.80, "#sqrt{s} = 13 TeV, 10 fb^{-1}")

c4.Draw()